In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from dython import nominal

In [3]:
df = pd.read_excel('seg_trans.xlsx')
seg=df.copy()
seg = seg.iloc[:,1:]
seg.head(10)
seg.isnull().sum()

First Policy´s Year                     30
Customer Age                            17
Educational Degree                      17
Gross Monthly Salary                    36
Geographic Living Area                   1
Has Children (Y=1)                      21
Customer Monetary Value                  0
Claims Rate                              0
Premiums in LOB: Motor                  34
Premiums in LOB: Household               0
Premiums in LOB: Health                 43
Premiums in LOB:  Life                 104
Premiums in LOB: Work Compensations     86
dtype: int64

In [4]:
seg.head()

,First Policy´s Year,Customer Age,Educational Degree,Gross Monthly Salary,Geographic Living Area,Has Children (Y=1),Customer Monetary Value,Claims Rate,Premiums in LOB: Motor,Premiums in LOB: Household,Premiums in LOB: Health,Premiums in LOB: Life,Premiums in LOB: Work Compensations
0,1985.0,34.0,1.0,4354.0,1.0,1.0,380.97,0.390032,375.85,79.45,146.36,47.01,16.89
1,1981.0,21.0,1.0,1353.0,4.0,1.0,-131.13,1.116503,77.46,416.20,116.69,194.48,106.13
2,1991.0,46.0,0.0,4554.0,3.0,0.0,504.67,0.284810,206.15,224.50,124.58,86.35,99.02
3,1990.0,35.0,2.0,2197.0,4.0,1.0,-16.99,0.986665,182.48,43.35,311.17,35.34,28.34
4,1986.0,43.0,2.0,3526.0,4.0,1.0,35.23,0.904281,338.62,47.80,182.59,18.78,41.45


In [5]:
def outlier_limits(col):
  Q3, Q1 = np.nanpercentile(col, [75, 25])
  IQR = Q3 - Q1
  UL = Q3 + 3*IQR
  LL = Q1 - 3*IQR
  return UL, LL

In [6]:
Q1 = seg.quantile(0.25)
Q3 = seg.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

First Policy´s Year                      12.000000
Customer Age                             30.000000
Educational Degree                        1.000000
Gross Monthly Salary                   3169.250000
Geographic Living Area                    3.000000
Has Children (Y=1)                        1.000000
Customer Monetary Value                 409.217500
Claims Rate                               0.584374
Premiums in LOB: Motor                  217.710000
Premiums in LOB: Household              240.600000
Premiums in LOB: Health                 108.020000
Premiums in LOB:  Life                   47.900000
Premiums in LOB: Work Compensations      46.120000
dtype: float64


In [23]:
Q1 - 5* IQR

First Policy´s Year                     1920.000000
Customer Age                            -117.000000
Educational Degree                        -4.000000
Gross Monthly Salary                  -12435.250000
Geographic Living Area                   -14.000000
Has Children (Y=1)                        -5.000000
Customer Monetary Value                -2055.527500
Claims Rate                               -2.529433
Premiums in LOB: Motor                  -897.960000
Premiums in LOB: Household             -1153.550000
Premiums in LOB: Health                 -428.300000
Premiums in LOB:  Life                  -229.610000
Premiums in LOB: Work Compensations     -219.930000
dtype: float64

In [24]:
Q3 + 5 * IQR

First Policy´s Year                     2052.000000
Customer Age                             213.000000
Educational Degree                         7.000000
Gross Monthly Salary                   22426.500000
Geographic Living Area                    19.000000
Has Children (Y=1)                         6.000000
Customer Monetary Value                 2445.865000
Claims Rate                                3.898682
Premiums in LOB: Motor                  1496.850000
Premiums in LOB: Household              1493.050000
Premiums in LOB: Health                  759.920000
Premiums in LOB:  Life                   297.290000
Premiums in LOB: Work Compensations      287.390000
dtype: float64

In [31]:
Q1 = seg.quantile(0.25)
Q3 = seg.quantile(0.75)
IQR = Q3 - Q1
df_w_out = seg[~((seg < (Q1 - 4.5* IQR)) |(seg > (Q3 + 4.5 * IQR))).any(axis=1)]
df_out= seg[((seg < (Q1 - 4.5* IQR)) |(seg > (Q3 + 4.5 * IQR))).any(axis=1)]


(10180, 13)

In [30]:
df_out.shape

(116, 13)

In [26]:
df_out
df_out.isnull().sum()


First Policy´s Year                    0
Customer Age                           0
Educational Degree                     0
Gross Monthly Salary                   0
Geographic Living Area                 0
Has Children (Y=1)                     0
Customer Monetary Value                0
Claims Rate                            0
Premiums in LOB: Motor                 1
Premiums in LOB: Household             0
Premiums in LOB: Health                1
Premiums in LOB:  Life                 1
Premiums in LOB: Work Compensations    0
dtype: int64

In [48]:
for column in ['Gross Monthly Salary','Customer Monetary Value','Claims Rate','Premiums in LOB: Motor','Premiums in LOB: Household','Premiums in LOB: Health','Premiums in LOB:  Life','Premiums in LOB: Work Compensations']:
    UL, LL = outlier_limits(df_2[column])
    df_2[column] = np.where((df_2[column] > UL) | (df_2[column] < LL), np.nan, df_2[column]) 

In [49]:
df_2.isnull().sum()

First Policy´s Year                     30
Customer Age                            17
Educational Degree                      17
Gross Monthly Salary                    38
Geographic Living Area                   1
Has Children (Y=1)                      21
Customer Monetary Value                 25
Claims Rate                             15
Premiums in LOB: Motor                  40
Premiums in LOB: Household             123
Premiums in LOB: Health                 47
Premiums in LOB:  Life                 245
Premiums in LOB: Work Compensations    248
dtype: int64

In [50]:
df_2.describe()

,First Policy´s Year,Customer Age,Educational Degree,Gross Monthly Salary,Geographic Living Area,Has Children (Y=1),Customer Monetary Value,Claims Rate,Premiums in LOB: Motor,Premiums in LOB: Household,Premiums in LOB: Health,Premiums in LOB: Life,Premiums in LOB: Work Compensations
count,10266.000000,10279.000000,10279.000000,10258.000000,10295.000000,10275.000000,10271.000000,10281.000000,10256.000000,10173.000000,10249.000000,10051.000000,10048.000000
mean,1991.062634,47.992217,1.478938,4996.326087,2.709859,0.706764,217.356589,0.679870,297.001526,194.821621,167.900408,38.966881,37.793692
std,511.267913,19.709476,0.795565,1969.053253,1.266291,0.455268,255.714406,0.319076,138.126269,208.608857,74.243510,40.759652,39.351664
min,1974.000000,15.000000,0.000000,665.000000,1.000000,0.000000,-490.200000,0.000000,-4.110000,-75.000000,-2.110000,-7.000000,-12.000000
25%,1980.000000,33.000000,1.000000,3411.000000,1.000000,0.000000,-9.220000,0.392249,190.590000,48.900000,111.800000,9.890000,10.000000
50%,1986.000000,48.000000,2.000000,5002.000000,3.000000,1.000000,187.040000,0.719227,298.500000,131.700000,162.810000,24.670000,25.340000
75%,1992.000000,63.000000,2.000000,6580.000000,4.000000,1.000000,399.080000,0.976113,407.687500,281.150000,219.040000,55.900000,53.900000
max,53784.000000,988.000000,3.000000,10042.000000,4.000000,1.000000,1571.760000,1.620145,585.220000,1011.850000,442.860000,201.370000,194.590000


In [54]:
drop_lines=[7195,9294]
df_2=df_2.drop(drop_lines)

In [55]:
df_2.describe()

,First Policy´s Year,Customer Age,Educational Degree,Gross Monthly Salary,Geographic Living Area,Has Children (Y=1),Customer Monetary Value,Claims Rate,Premiums in LOB: Motor,Premiums in LOB: Household,Premiums in LOB: Health,Premiums in LOB: Life,Premiums in LOB: Work Compensations
count,10264.000000,10277.000000,10277.000000,10256.000000,10293.000000,10273.000000,10269.000000,10279.000000,10254.000000,10171.000000,10247.000000,10049.000000,10046.000000
mean,1986.016368,47.898803,1.478836,4996.111447,2.709900,0.706902,217.350731,0.679891,296.983351,194.839500,167.904265,38.972214,37.797090
std,6.612072,17.392619,0.795609,1969.175749,1.266238,0.455205,255.735047,0.319091,138.132546,208.621746,74.248220,40.761697,39.354822
min,1974.000000,15.000000,0.000000,665.000000,1.000000,0.000000,-490.200000,0.000000,-4.110000,-75.000000,-2.110000,-7.000000,-12.000000
25%,1980.000000,33.000000,1.000000,3411.000000,1.000000,0.000000,-9.220000,0.392272,190.590000,48.900000,111.800000,9.890000,10.000000
50%,1986.000000,48.000000,2.000000,5001.000000,3.000000,1.000000,187.040000,0.719227,298.500000,131.700000,162.810000,24.670000,25.340000
75%,1992.000000,63.000000,2.000000,6580.000000,4.000000,1.000000,399.080000,0.976125,407.520000,281.150000,219.040000,55.900000,53.900000
max,1998.000000,81.000000,3.000000,10042.000000,4.000000,1.000000,1571.760000,1.620145,585.220000,1011.850000,442.860000,201.370000,194.590000


In [58]:
df_2.isnull().sum()
df_2.to_excel('C:\\Users\\FilipeSousa\\Desktop\\Challenge-Closer-Academy-\\Project\\data_outliers_As_na.xlsx')

In [42]:
from scipy import stats
seg_o=seg.iloc[:,[3,6,7,8,9,10,11,12]]
seg_o.head()

,Gross Monthly Salary,Customer Monetary Value,Claims Rate,Premiums in LOB: Motor,Premiums in LOB: Household,Premiums in LOB: Health,Premiums in LOB: Life,Premiums in LOB: Work Compensations
0,4354.0,380.97,0.390032,375.85,79.45,146.36,47.01,16.89
1,1353.0,-131.13,1.116503,77.46,416.20,116.69,194.48,106.13
2,4554.0,504.67,0.284810,206.15,224.50,124.58,86.35,99.02
3,2197.0,-16.99,0.986665,182.48,43.35,311.17,35.34,28.34
4,3526.0,35.23,0.904281,338.62,47.80,182.59,18.78,41.45


In [43]:
z=np.abs(stats.zscore(seg_o))
print(z)

       Gross Monthly Salary  Customer Monetary Value  Claims Rate  \
0                       NaN                 0.104371     0.120937   
1                       NaN                 0.158822     0.128126   
2                       NaN                 0.167947     0.157012   
3                       NaN                 0.100160     0.083612   
4                       NaN                 0.073321     0.055367   
...                     ...                      ...          ...   
10291                   NaN                 0.091484     0.074883   
10292                   NaN                 0.630979     0.254656   
10293                   NaN                 0.177933     0.182202   
10294                   NaN                 0.037085     0.033504   
10295                   NaN                 0.146916     0.162075   

       Premiums in LOB: Motor  Premiums in LOB: Household  \
0                         NaN                    0.371495   
1                         NaN                    

In [27]:
df_out=seg[(z >3).all(axis=1)]


In [28]:
df_out.head()

,First Policy´s Year,Customer Age,Educational Degree,Gross Monthly Salary,Geographic Living Area,Has Children (Y=1),Customer Monetary Value,Claims Rate,Premiums in LOB: Motor,Premiums in LOB: Household,Premiums in LOB: Health,Premiums in LOB: Life,Premiums in LOB: Work Compensations
